(concepts:errors)=
# Error handling in ``configpile``

The goal behind ``configpile`` error reporting is to provide helpful error messages to the user. In
particular:

- ``configpile`` does not rely on Python exceptions, rather implements its own error class

- The error class is designed to be used in a 
  [result type](https://en.wikipedia.org/wiki/Result_type) that follows existing Python usage
  patterns. (To be pedantic, it is not monadic.)

- ``configpile`` accumulate errors instead of stopping at the first error

- Instead of relying on stack traces to convey contextual information, ``configpile`` errors
  store context information that is manually added when results are processed.

## Errors

The base error type is `Err`, which contains either a single error or a sequence of errors.

A single error is constructed through the {meth}`~configpile.userr.Err.make` static method.

Errors can be pretty-printed. If the [Rich](https://github.com/Textualize/rich) library is available, some light formatting will be applied.

In [1]:
from configpile import Err
e1 = Err.make("First error", context_info = 1, other_info = "bla")
e1.pretty_print()

In other_info: blub                                                                          

In context_info: 1                                                                           

First error

Errors can be collected in a single {class}`~configpile.userr.Err` instance, and pretty-printing will collect errors occurring in the same context.

In [2]:
e1 = Err.make("First error", context_info = 1, other_info = "blub")
e2 = Err.make("Second error", context_info = 1)
e12 = Err.collect1(e1, e2)
e12.pretty_print()

 • In context_info: 1                                                                        
    0 In other_info: blub                                                                    
      First error                                                                            
    1 Second error

A sequence of single errors can always be recovered:

In [3]:
e12.errors()

## Results

The error type is designed to be used in functions that either return a valid value, or an error.
Such functions return a result, or a {data}`configpile.userr.Res` type.

Note that the {data}`configpile.userr.Res` type is parameterized by the valid value type:
in the example below, it is {class}`int`.

An example of such a function would be:

In [4]:
from configpile.userr import Res

def parse_int(s: str) -> Res[int]:
    try:
        return int(s)
    except ValueError as e:
        return Err.make(str(e))

and would give the following results:

In [5]:
parse_int("invalid")

Err1("invalid literal for int() with base 10: 'invalid'")

In [6]:
parse_int(1234)

1234

Results can be processed further. For example, the function that squares the value contained in a
result, while leaving any error untouched, can be written:

In [7]:
def square_result(res: Res[int]) -> Res[int]:
    if isinstance(res, Err):
        return res
    return res*res

... or, using the {func}`~configpile.userr.map` helper:

In [8]:
from configpile import userr

def square_result1(res: Res[int]) -> Res[int]:
    return userr.map(lambda x: x*x, res)

and we have, unsurprisingly:

In [9]:
square_result(parse_int("invalid"))

Err1("invalid literal for int() with base 10: 'invalid'")

In [10]:
square_result1(parse_int(4))

16

The {func}`~configpile.userr.flat_map` function is useful to chain processing where each step can fail.

In [27]:
import math

def square_root(x: int) -> Res[float]:
    if x < 0:
        return Err.make(f"Cannot take square root of negative number {x}")
    else:
        return math.sqrt(float(x))

In [28]:
userr.flat_map(square_root, parse_int("valid"))

Err1("invalid literal for int() with base 10: 'valid'")

In [29]:
userr.flat_map(square_root, parse_int("2"))

1.4142135623730951

In [31]:
userr.flat_map(square_root, parse_int("-2"))

Err1('Cannot take square root of negative number -2')

## Combining results and errors

Finally, the {mod}`~configpile.userr` module offers ways to combine results.

For example, if one parses several integers, one can collect the results in a tuple using the
{func}`configpile.userr.collect` function.

In [14]:
userr.collect(parse_int(2), parse_int(3))

(2, 3)

In [15]:
userr.collect(parse_int(3), parse_int("invalid"))

Err1("invalid literal for int() with base 10: 'invalid'")

In [16]:
userr.collect(parse_int("invalid"), parse_int("invalid"))

ManyErr(errs=[Err1("invalid literal for int() with base 10: 'invalid'"), Err1("invalid literal for int() with base 10: 'invalid'")])

See also {func}`configpile.userr.collect_seq`  when dealing with sequences.

Errors can be collected and combined too. The {meth}`configpile.userr.Err.collect1` method expect 
at least one argument and returns an {class}`~configpile.userr.Err`, while 
{meth}`configpile.userr.Err.collect` can deal with no argument being passed, or with optional
arguments.

In particular, optional errors, of type `Optional[Err]`, are great for validation: a {data}`None`
value indicates no error, while an error indicates that one or several problems are present.

In [20]:
from typing import Optional, Sequence
a = -2
b = 1
check_a: Optional[Err] = Err.check(a > 0, "a must be positive")
check_b: Optional[Err] = Err.check(b > 0, "b must be positive")
Err.collect(check_a, check_b)

Err1('a must be positive')